In [176]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, execute
from qiskit_ibm_provider import IBMProvider
import qiskit
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from QuAwesome import DataManager
import qutip
from qutip import Qobj
from qutip import ptrace

In [12]:
# set up the ibmq account 

# only exceute once , it will load automatically afterward
token="5e323c289cb41e03e565a0c882fcf8a92d44d7ff175b755a59b982629216c24369583d3311ae92751dc459f5e257f9ee4e2c26e03c738db142ecb296564a87b9"
IBMProvider.save_account(token=token,overwrite=True)
provider = IBMProvider()

In [170]:

import collections
from qutip import Qobj

cvt = {'0' : 1 , '1' : -1}
def get_exp(k , iid):
    rs = 1
    for idx ,i in enumerate(k[::-1]) :
        if idx in iid :
            rs = rs
        else :
            rs = rs * cvt[i]
    return rs
def calculate_exp(source , iid):
    rs = 0
    shots = sum([v for v in source.values()])
    for k,v in source.items() :
        rs += get_exp(k , iid) * (v/shots)
    return rs
def Tomography(raw_counts) :
    # X Y Z
    #print("======")
    iid = Qobj(np.array([[1, 0], [0, 1]])) # I
    pauli = [
        Qobj(np.array([[0, 1], [1, 0]])) , # X
        Qobj(np.array([[0, -1j], [1j, 0]])) , # Y
        Qobj(np.array([[1, 0], [0, -1]])) , # Z
        iid # I
    ]
    name = ['X' ,'Y' ,'Z' , "I"]
   
    rho = qutip.tensor(iid , iid ,iid) # I*I*I
    ctrl = [
        [0] ,
        [0] ,
        [0,1]
    ]
    for ql0 in range(4) :
        q0 = ql0
        if ql0 ==  3 :
            q0 = 2
        for ql1 in range(4) :
            q1 = ql1
            if ql1 == 3 :
                q1 = 2
            for ql2 in range(4):
                q2 = ql2
                if ql2 == 3 :
                    q2 = 2
                pauli_mt  = qutip.tensor(pauli[ql0] , pauli[ql1] , pauli[ql2])
                tg = dict(raw_counts[q0][q1][q2])
                qiid = [idx for idx ,i in enumerate([ql0 ,ql1 ,ql2]) if i == 3 ]
                v = calculate_exp(tg , qiid)
                #print(name[ql0] +name[ql1] + name[ql2] , v)
                rho += pauli_mt * v
                
    rho -= qutip.tensor(iid , iid ,iid)
    #print(rho.tr())
    return rho*0.125

In [350]:
iid = Qobj(np.array([[1, 0], [0, 1]])) # I
EEE = Qobj(np.array([[0, 0], [0, 1]])) # I
GGG = Qobj(np.array([[1, 0], [0, 0]])) # I
rho = qutip.tensor(EEE,GGG,EEE) # I*I*I

In [351]:
rho

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]

# Quantum Circuit

In [75]:
# define the physical qubit you wanna use
# choose the qubit on the ibmq page
# q = [9,62,65] 
# qMax = max(q) + 1
qMax = 3
# register the qubit and classical bit
Q = QuantumRegister(qMax, name='q')
# C = ClassicalRegister(2, name='bell')

In [336]:
# Generate 3-qubit GHZ state
qc = qiskit.QuantumCircuit(Q)
qc.h(0)
qc.cx(0, (1,2))

qc.barrier(0,1,2)
qc.h(0)
qc.cu(0,np.pi/2,0,0,0, 1)
qc.cu(0,np.pi/4,0,0,0, 2)
qc.h(1)
qc.cu(0,np.pi/2,0,0,1, 2)
qc.h(2)

In [338]:
print(qc)

     ┌───┐           ░ ┌───┐                                     »
q_0: ┤ H ├──■────■───░─┤ H ├───────■───────────────■─────────────»
     └───┘┌─┴─┐  │   ░ └───┘┌──────┴───────┐       │        ┌───┐»
q_1: ─────┤ X ├──┼───░──────┤ U(0,π/2,0,0) ├───────┼────────┤ H ├»
          └───┘┌─┴─┐ ░      └──────────────┘┌──────┴───────┐└───┘»
q_2: ──────────┤ X ├─░──────────────────────┤ U(0,π/4,0,0) ├─────»
               └───┘ ░                      └──────────────┘     »
«                          
«q_0: ─────────────────────
«                          
«q_1: ───────■─────────────
«     ┌──────┴───────┐┌───┐
«q_2: ┤ U(0,π/2,0,0) ├┤ H ├
«     └──────────────┘└───┘


In [276]:
# q=[18,19,20]
q=[0,1,2]
qMax = max(q) + 1
Q = QuantumRegister(qMax, name='q')
c_bell = ClassicalRegister(3, name='tomography')

qcs = []
for ql0 in range(3) :
    for ql1 in range(3) :
        for ql2 in range(3):
            qc = QuantumCircuit(Q, c_bell)

            qc.h(Q[q[0]])
            qc.cx(Q[q[0]], (Q[q[1]],Q[q[2]]))
            # qc.barrier(0,1,2)
            qc.h(Q[q[0]])
            qc.cu(0,np.pi/2,0,0,Q[q[0]], Q[q[1]])
            qc.cu(0,np.pi/4,0,0,Q[q[0]], Q[q[2]])
            qc.h(Q[q[1]])
            qc.cu(0,np.pi/2,0,0,Q[q[1]], Q[q[2]])
            qc.h(Q[q[2]])

            

            qc.barrier(range(qMax))
            # tomography
            if ql0 == 0 :     # measure x
                qc.h(Q[q[0]])
            if ql0 == 1 :     # measure y
                qc.sdg(Q[q[0]])
                qc.h(Q[q[0]])
                
            if ql1 == 0 :     # measure x
                qc.h(Q[q[1]])
            if ql1 == 1 :     # measure y
                qc.sdg(Q[q[1]])
                qc.h(Q[q[1]])
                
            if ql2 == 0 :     # measure x
                qc.h(Q[q[2]])
            if ql2 == 1 :     # measure y
                qc.sdg(Q[q[2]])
                qc.h(Q[q[2]])
            qc.measure([Q[q[0]], Q[q[1]] , Q[q[2]]], [c_bell[0], c_bell[1] , c_bell[2]])
            # transpile qc
            qcs.append(qc)

In [277]:
print(qcs[1])

              ┌───┐          ┌───┐                                     »
         q_0: ┤ H ├──■────■──┤ H ├───────■───────────────■─────────────»
              └───┘┌─┴─┐  │  └───┘┌──────┴───────┐       │        ┌───┐»
         q_1: ─────┤ X ├──┼───────┤ U(0,π/2,0,0) ├───────┼────────┤ H ├»
                   └───┘┌─┴─┐     └──────────────┘┌──────┴───────┐└───┘»
         q_2: ──────────┤ X ├─────────────────────┤ U(0,π/4,0,0) ├─────»
                        └───┘                     └──────────────┘     »
tomography: 3/═════════════════════════════════════════════════════════»
                                                                       »
«                                    ░  ┌───┐      ┌─┐      
«         q_0: ──────────────────────░──┤ H ├──────┤M├──────
«                                    ░  ├───┤      └╥┘┌─┐   
«         q_1: ───────■──────────────░──┤ H ├───────╫─┤M├───
«              ┌──────┴───────┐┌───┐ ░ ┌┴───┴┐┌───┐ ║ └╥┘┌─┐
«         q_2: ┤ U(0,π/2,0,0) ├┤ H ├─░

# Local Simulation

In [278]:

# Construct an ideal simulator
aersim = AerSimulator(shots=10000)

# Perform an ideal simulation
result_ideal = aersim.run(qcs).result()
# counts_ideal = result_ideal.get_state()
# print('Counts(ideal):', counts_ideal)
# Counts(ideal): {'000': 493, '111': 531}

# Counts(noise): {'101': 16, '110': 48, '100': 7, '001': 31, '010': 7, '000': 464, '011': 15, '111': 436}

In [279]:
t = result_ideal.get_counts()
shape=(3,3,3)
count_list = np.reshape(np.array(t) ,shape)

In [343]:
ρ=Tomography(count_list)
ρ

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 2.50900e-01+0.j       -3.50000e-04+0.0032j    1.24750e-01+0.1245j
   1.28600e-01-0.126725j  2.14450e-01+0.08245j   3.87250e-02-0.090325j
   3.99250e-02+0.08595j   2.15025e-01-0.0888j  ]
 [-3.50000e-04-0.0032j    0.00000e+00+0.j        3.05000e-03+0.001725j
  -1.85000e-03-0.00355j  -2.02500e-03-0.001325j  1.00000e-04+0.0013j
  -9.75000e-04+0.00395j   1.92500e-03+0.00385j ]
 [ 1.24750e-01-0.1245j    3.05000e-03-0.001725j  1.24200e-01+0.j
   1.10000e-03-0.1305j    1.49075e-01-0.06325j  -2.65250e-02-0.0605j
   6.23000e-02+0.0265j    6.05750e-02-0.150125j]
 [ 1.28600e-01+0.126725j -1.85000e-03+0.00355j   1.10000e-03+0.1305j
   1.21200e-01+0.j        5.84750e-02+0.15245j   6.15750e-02-0.02495j
  -2.71750e-02+0.062375j  1.50750e-01+0.06295j ]
 [ 2.14450e-01-0.08245j  -2.02500e-03+0.001325j  1.49075e-01+0.06325j
   5.84750e-02-0.15245j   2.12400e-01+0.j        3.00000e-03-0.0877j
   6.19500

# Real device

In [281]:
provider.backends()

[<IBMBackend('ibm_kyoto')>,
 <IBMBackend('ibm_osaka')>,
 <IBMBackend('ibm_sherbrooke')>,
 <IBMBackend('ibm_brisbane')>]

In [282]:
# select the machine ; quantum device or local simulator
# check out homepage of ibmq for accessible device
machine = 'ibm_osaka'
backend = provider.get_backend(machine)

## Execute

In [283]:
shot=10000
data = {
    'backend': None,
    'job_id' : None,
    'q_list' : [],
    'shots'  : shot,
    'Robust' : None,
    'Signal' : None,
    'assemb' : None,
    'counts' : [],
    'Memory' : []
}
DM = DataManager(data)

In [284]:
job = execute(qcs, backend=backend, shots=shot, optimization_level=0, memory=True)

In [285]:

DM['job_id'] = job.job_id()
DM['job_id']

'csevcmjdape000862xz0'

In [286]:
FILENAME = "testtest"
DM.save(FILENAME, False)

Save data to 'testtest.json' (success)


In [292]:
job.status()

<JobStatus.DONE: 'job has successfully run'>

# Result

In [293]:
result = job.result()
DM['counts'] = result.get_counts()


In [342]:
DM['counts'][3]

{'011': 451,
 '000': 2288,
 '010': 1687,
 '100': 945,
 '101': 466,
 '111': 707,
 '110': 2972,
 '001': 484}

In [294]:
shape=(3,3,3)
count_list_real = np.reshape(np.array(DM['counts']) ,shape)

ρ=Tomography(count_list_real)
ρ

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 0.173   +0.j        0.01645 -0.00115j   0.0699  +0.04365j
   0.01915 -0.045375j  0.08    +0.0415j    0.023425-0.014575j
   0.003725+0.047275j  0.065325-0.083j   ]
 [ 0.01645 +0.00115j   0.0646  +0.j        0.04695 -0.032325j
   0.0041  -0.01375j   0.034175-0.023925j  0.02125 +0.00315j
   0.008075-0.0122j    0.021625-0.011075j]
 [ 0.0699  -0.04365j   0.04695 +0.032325j  0.1647  +0.j
  -0.02435 -0.0451j    0.088225-0.018125j -0.009175-0.02455j
   0.04095 +0.0305j    0.044975-0.058625j]
 [ 0.01915 +0.045375j  0.0041  +0.01375j  -0.02435 +0.0451j
   0.0724  +0.j        0.040475+0.04705j   0.026925+0.015325j
  -0.007375+0.015225j  0.0425  +0.04445j ]
 [ 0.08    -0.0415j    0.034175+0.023925j  0.088225+0.018125j
   0.040475-0.04705j   0.1897  +0.j        0.00275 -0.0306j
   0.03955 +0.0446j    0.08035 -0.063825j]
 [ 0.023425+0.014575j  0.02125 -0.00315j  -0.009175+0.02455j
   0.026925-0.0